In [ ]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
import pandas as pd

import sys
current_working_dir = os.getcwd()
# get the path to directory "pe_granular_analysis"
_path = current_working_dir[:current_working_dir.find("pe_granular_analysis")+len("pe_granular_analysis")]
if _path not in sys.path:
    sys.path.append(_path)

from preprocessing.nifti_conversion import batch_convert_dicom_to_volume
from preprocessing.dataframe_formatter import dataframe_norm
from preprocessing.segmentation import segmentator_process, CustomSegmentator
from preprocessing.segmentation import segmentation2bbox_batch_process
from preprocessing.dataframe_formatter import add_path2df

from configs.pe.pe_final import PELabelMask

from explainer.utils.visualize_cam import overlap_cam_on_voxel
from utils.data_visualizer import ColorImageSliceViewer3D
from utils.base import format_input_path

from tqdm import tqdm

import gc

import numpy as np
import torch
torch.backends.cudnn.enabled = False

In [ ]:
# build/load your dataframe here
example_data_dir = os.path.join(_path, "data")
example_output_dir = os.path.join(_path, "temp")

example_pid = None
example_study_uid_1 = "7f6fb39566ed"
example_series_uid_1 = "b34edb1a4de7"

example_study_uid_2 = "00268ff88746"
example_series_uid_2 = "75d23269adbd"

df = pd.DataFrame([
    {
        "PatientID": None,
        "StudyInstanceUID": example_study_uid_1,
        "SeriesInstanceUID": example_series_uid_1,
    },
    {
        "PatientID": None,
        "StudyInstanceUID": example_study_uid_2,
        "SeriesInstanceUID": example_series_uid_2,
    },
])

In [ ]:
err_suids = batch_convert_dicom_to_volume(
    df=df,
    source_folder=example_data_dir,
    output_folder=example_output_dir,
)
if len(err_suids) > 0:
    print("Errors occurred during DICOM to volume conversion:")
    for error in err_suids:
        print(error)
else:
    print("DICOM to volume conversion completed successfully.")

In [ ]:
cs = CustomSegmentator(task="lung", device="cuda")
for _, row in tqdm(df.iterrows(), total=df.shape[0]):
    _pid = row["PatientID"] if "PatientID" in row else None
    _study_uid = row["StudyInstanceUID"] if "StudyInstanceUID" in row else None
    _series_uid = row["SeriesInstanceUID"] if "SeriesInstanceUID" in row else None
    error = segmentator_process(
        segmentator_instance=cs,
        input_folder=example_output_dir,
        pid=_pid,
        study_uid=_study_uid,
        series_uid=_series_uid,
    )
    if error is not None:
        print(f"Error occurred during segmentation for SeriesInstanceUID: {_series_uid}")

In [ ]:
df = segmentation2bbox_batch_process(df, task="lung", input_folder=example_output_dir, save_path=example_output_dir)
df = add_path2df(df, path=example_output_dir)
df.to_csv(os.path.join(example_output_dir, "lung_labels_bbox_with_path.csv"), index=False)
df

In [ ]:
cfg = PELabelMask()
cfg.ddp = False
cfg.img_size = 256 # 256 used in training; 384 vs 256 depends on your GPU memory
cfg.working_dir = example_output_dir
cfg.test_result_dir = example_output_dir
cfg.on_deploy = True
cfg.on_grad_cam = False
print(cfg)

In [ ]:
dataset_instance = cfg.dataset_class(dataframe=df, usage="inference", config=cfg)
data_loader = torch.utils.data.DataLoader(
    dataset_instance,
    batch_size=1, # or more if memory allows
    shuffle=False,
    num_workers=0, # set to 0 for debugging, increase for speed if needed
    pin_memory=True,
    drop_last=False,
)

In [ ]:
model_settings = cfg.get_model_setups()

weight_paths = [os.path.join(
    _path,
   "models/weights/example_weight.pth"
)]
# define your weight paths here if you have multiple checkpoints
# e.g., weight_paths = [path1, path2, path3]
# weight_paths = []
# for fold in range(5):
#     weight_paths.append(os.path.join(
#         _path,
#        f"models/weights/pe_coatnet_224_fold{fold}.pth"
#     ))

models = []
for weights_path in weight_paths:
    model = cfg.model_class(**model_settings)
    model.custom_load_from_checkpoint(weights_path)
    model = model.cuda()
    model.eval()
    models.append(model)

In [ ]:
predictions = []
with torch.no_grad():
    for batch in data_loader:
        input_images = batch["images"].cuda()
        # if multiple models, average their outputs
        for i, model in enumerate(models):
            outputs = model(input_images)
            if i == 0:
                prediction = torch.sigmoid(outputs).cpu().numpy()
            else:
                prediction += torch.sigmoid(outputs).cpu().numpy()
        prediction /= len(models)
        predictions.append(prediction)

# flatten the list of predictions
predictions = np.vstack(predictions).reshape(-1)
# attach predictions to the dataframe
df['pe_present_in_exam_pred'] = predictions
df

In [ ]:
# if you need to visualize some CAMs, you can use the following code

# from explainer.explain_pipeline import explain
# from explainer.configs.explainer_cfg import ExplainerConfig

# explain_cfg = ExplainerConfig(dev_cfg=cfg)

# model_weight_paths = os.path.join(
#     _path,
#    "models/weights/ultimate.pe.aug.pth"
# )
# cases_to_explain = df
# test_name = "cam_test"

# explain(cfg=explain_cfg, model_weight_paths=model_weight_paths, cases_to_explain=cases_to_explain, test_name=test_name)